In [104]:
import random

In [105]:
# Define the graph representing the train network
graph = {
    'EL ROSARIO': {'INSTITUTO DEL PETROLEO': 6, 'TACUBA': 4},
    'INSTITUTO DEL PETROLEO': {'EL ROSARIO': 6, 'DEPORTIVO 18 DE MARZO': 2, 'LA RAZA': 2},
    'DEPORTIVO 18 DE MARZO': {'INSTITUTO DEL PETROLEO': 2,'LA RAZA': 2, 'MARTIN CARRERA': 2},
    'MARTIN CARRERA': {'DEPORTIVO 18 DE MARZO': 2, 'CONSULADO': 2},
    'LA RAZA': {'DEPORTIVO 18 DE MARZO': 2, 'INSTITUTO DEL PETROLEO': 2, 'GUERRERO': 2, 'CONSULADO': 3},
    'CONSULADO': {'LA RAZA': 3, 'MARTIN CARRERA': 2, 'OCEANIA': 3, 'MORELOS': 2},
    'TACUBA': {'EL ROSARIO': 4, 'HIDALGO': 7, 'TACUBAYA': 5},
    'GUERRERO': {'LA RAZA': 2, 'HIDALGO': 1, 'GARIBALDI': 1},
    'GARIBALDI': {'MORELOS': 3, 'BELLAS ARTES': 1, 'GUERRERO': 1},
    'OCEANIA': {'CONSULADO': 3, 'SAN LAZARO': 3, 'PANTITLAN': 3},
    'MORELOS': {'GARIBALDI': 3, 'CONSULADO': 2, 'SAN LAZARO': 1, 'CANDELARIA': 1},
    'HIDALGO': {'TACUBA': 7, 'GUERRERO': 1, 'BELLAS ARTES': 1, 'BALDERAS': 2},
    'BELLAS ARTES': {'HIDALGO': 1, 'SALTO DEL AGUA': 2, 'GARIBALDI': 1, 'PINO SUAREZ': 3},
    'SAN LAZARO': {'MORELOS': 1, 'CANDELARIA': 1, 'PANTITLAN': 6, 'OCEANIA': 3},
    'BALDERAS': {'TACUBAYA': 6, 'HIDALGO': 2, 'SALTO DEL AGUA': 1, 'CENTRO MEDICO': 3},
    'SALTO DEL AGUA': {'BELLAS ARTES': 2, 'BALDERAS': 1, 'PINO SUAREZ': 2, 'CHABACANO': 3},
    'PINO SUAREZ': {'BELLAS ARTES': 3, 'SALTO DEL AGUA': 2, 'CANDELARIA': 2, 'CHABACANO': 1},
    'CANDELARIA': {'SAN LAZARO': 1, 'PINO SUAREZ': 2, 'MORELOS': 1, 'JAMAICA': 2},
    'JAMAICA': {'CANDELARIA': 2, 'CHABACANO': 1, 'PANTITLAN': 5, 'SANTA ANITA': 1},
    'CHABACANO': {'JAMAICA': 1, 'PINO SUAREZ': 1, 'SALTO DEL AGUA': 3, 'CENTRO MEDICO': 2, 'SANTA ANITA': 2, 'ERMITA': 6},
    'CENTRO MEDICO': {'CHABACANO': 2, 'BALDERAS': 3, 'TACUBAYA': 3, 'ZAPATA': 4},
    'TACUBAYA': {'CENTRO MEDICO': 3, 'BALDERAS': 6, 'TACUBA': 5, 'MIXCOAC': 3},
    'SANTA ANITA': {'CHABACANO': 2, 'JAMAICA': 1, 'ATLALILCO': 6},
    'MIXCOAC': {'TACUBAYA': 3, 'ZAPATA': 3},
    'ZAPATA': {'MIXCOAC': 3, 'CENTRO MEDICO': 4, 'ERMITA': 3},
    'ERMITA': {'ZAPATA': 3, 'CHABACANO': 6, 'ATLALILCO': 2},
    'ATLALILCO': {'ERMITA': 2, 'SANTA ANITA': 6},
    'PANTITLAN': {'JAMAICA': 5, 'SAN LAZARO': 6, 'OCEANIA': 3}

}



In [106]:
# Define the starting and destination stations
start_station = 'EL ROSARIO'
destination_station = 'SAN LAZARO'

# Define genetic algorithm parameters
tournament_size = 100# cantidad de rutas que se van a seleccionar para competir
population_size = 200#la cantidad de rutas posibles
max_generations = 100
mutation_rate = 1/population_size

In [107]:


def generate_initial_population(population_size):
    population = []
    for _ in range(population_size):
        route = [start_station]
        current_station = start_station
        while current_station != destination_station: # The loop ends when it reaches the destination station
            next_station = random.choice(list(graph[current_station].keys())) # Select stations connected to the current station randomly
            route.append(next_station) # Add the next station to the route list
            current_station = next_station
        population.append(route) # Add the new route to the list
    return population


In [108]:
def fitness(route):
    # Función de aptitud que calcula el costo total de una ruta (distancia total)
    total_cost = 0
    for i in range(len(route) - 1):
        # Calcula la distancia entre cada par de estaciones consecutivas en la ruta
        if route[i+1] in graph[route[i]]:#si la siguiente estacion se encuentra en el grafo asociado a la estacion actual se suma la distancia entre nodos
            total_cost += graph[route[i]][route[i+1]]
            
        else:
            total_cost += 999  # Valor alto para penalizar rutas no válidas
    return total_cost

In [109]:
def tournament_selection(population, tournament_size):
    selected_parents = []
    while len(selected_parents) < population_size // 2:#mientras que la cantidad de rutas optimas sea menor que le mitad de la cantidad de rutas totales
        tournament = random.sample(population, tournament_size)#selecciona n rutas, donde n = torunament_size
        winner = min(tournament, key=fitness)#selecciona la ruta en donde se tenga la menor tiempo
        selected_parents.append(winner)#se van agrupando a las mejores rutas
    return selected_parents

def crossover(parent1, parent2):#cruazamiento de un punto
    crossover_point = random.randint(1, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child


def mutate(route):
    mutated_route = route[:]
    for i in range(1, len(mutated_route) - 1):
        if random.random() < mutation_rate:#si la mutatuion_rate es menor que un numero aleatorio entre cero y un se ejecuta la mutacion 
            current_station = mutated_route[i]
            if current_station != destination_station:#la estacion final siempre debe de ser diferente a la estacion que se va a cambiar
                possible_next_stations = [station for station in graph[current_station] if station != destination_station and station not in mutated_route[:i]]
                if possible_next_stations:
                    new_station = random.choice(possible_next_stations)
                    mutated_route[i] = new_station
            else:
                # Ensure the destination station is included in the route
                mutated_route[-1] = destination_station
    return mutated_route

In [110]:
# Genetic Algorithm 
population = generate_initial_population(population_size)
for generation in range(max_generations):
    parents = tournament_selection(population, tournament_size)
    #best_route = min(population, key=fitness)
    #print(f"Generation {generation}: Best Route - {best_route}, Total Cost - {fitness(best_route)}")
    new_population = []
    while len(new_population) < population_size:
        parent1 = random.choice(population)
        parent2 = random.choice(population)
        child = crossover(parent1, parent2)
        child = mutate(child)
        new_population.append(child)
    population.extend(new_population)#agrega las nuevas rutas 

best_route = min(population, key=fitness)
print("Best Route:", best_route)
print("Total Cost:", fitness(best_route))


Best Route: ['EL ROSARIO', 'INSTITUTO DEL PETROLEO', 'DEPORTIVO 18 DE MARZO', 'LA RAZA', 'CONSULADO', 'MORELOS', 'SAN LAZARO']
Total Cost: 16
